# Dongarra et al. (1996) D2 method
$n$次チェビシェフ多項式$T_n$の$0\leq n\leq N-4$について, 
\begin{equation}
\begin{split}
&\dfrac{1}{24} \sum^{N}_{\substack{p=n+4\\ p\equiv n\; (\mathrm{mod} \; 2)}}{p\left[p^2(p^2-4)^2-3n^2p^4+3n^4p^2-n^2\left(n^2-4 \right) ^2-48\alpha ^2(p^2-n^2) \right] a_p} -8\alpha ^2(n+2)(n+1)a_{n+2} +\alpha ^4c_na_n \\
&\quad -\dfrac{i\alpha R}{2} \left[\sum^{N}_{p=2}{a_p\sum^{}_{\substack{m\equiv p\; (\mathrm{mod} \; 2) \\ |m|\leq p-2 \\ |n-m|\leq N}}{p(p^2-m^2)c_{|n-m|}b_{|n-m|} }} -\alpha ^2\sum^{}_{\substack{|p|\leq N \\ |n-p|\leq N}}{c_{|p|}a_{|p|}c_{|n-p|}b_{|n-p|}} -\sum^{}_{\substack{|p|\leq N \\ |n-p|\leq N}}{c_{|p|}a_{|p|}\sum^{N}_{\substack{m=|n-p|+2\\ m+n\equiv p\; (\mathrm{mod} \; 2)}}{m\left[m^2-(n-p)^2 \right] b_m}} \right] \\
&\quad +\lambda \left\{i\alpha R\sum^{N}_{\substack{p=n+2\\ p\equiv n\; (\mathrm{mod} \; 2)}}{p(p^2-n^2)a_p} -i\alpha ^3Rc_na_n \right\} =0.
\end{split}
 \label{eq:Orszag1971-17-A44}
\end{equation}
および境界条件から得られる係数の関係式:
$$
\sum^{N_{\mathrm{e}}}_{\substack{p=0\\ p=0\; (\mathrm{mod}\; 2)}}{a_{\upsilon ,p}} =0,\quad 
\sum^{N_{\mathrm{e}}}_{\substack{p=0\\ p=0\; (\mathrm{mod}\; 2)}}{a_{\chi ,p}} =0,\quad 
\sum^{N_{\mathrm{o}}}_{\substack{p=1\\ p=1\; (\mathrm{mod}\; 2)}}{a_{\upsilon ,p}} =0,\quad 
\sum^{N_{\mathrm{o}}}_{\substack{p=1\\ p=1\; (\mathrm{mod}\; 2)}}{a_{\chi ,p}} =0.
$$
求める固有値方程式は
$$
(A-B\lambda )\textbf{x} =\textbf{0} \quad \rightarrow \quad A\textbf{x} =\lambda B\textbf{x} ,
$$
$$
\textbf{x} \equiv \left[a_{\upsilon ,0},a_{\upsilon ,1},\cdots ,a_{\upsilon ,N},\cdots a_{\chi ,0},a_{\chi ,1},\cdots ,a_{\chi ,N} \right] ^T.
$$

In [7]:
#using .Lilly1966_solver
using OffsetArrays  # 配列要素番号を任意開始にするモジュール (行列計算時は 1 始まりの通常 matrix に copy する)
using LinearAlgebra

#-- Drawing parameters    
N = 200  # wavenumber in Chebyshev
Ncal = N + 1
Ntot = 2 * (N + 1)

# initialize working arrays
# A: non-lambda coefficients, B: lambda coefficients
A_J = reshape(zeros(Complex{Float64},Ntot,Ntot),Ntot,Ntot)  # Matrix for calculation
B_J = reshape(zeros(Complex{Float64},Ntot,Ntot),Ntot,Ntot)  # Matrix for calculation
### --- From here, supposing matrix element as row, col as in mathematical notation
Acal = reshape(zeros(Complex{Float64},Ntot,Ntot),Ntot,Ntot)  # Matrix for calculation
Bcal = reshape(zeros(Complex{Float64},Ntot,Ntot),Ntot,Ntot)  # Matrix for calculation
A_vv = OffsetArray(Acal[1:Ncal,1:Ncal],0:N,0:N)  # coefficient matrix for v in v Chebyshev
A_vchi = OffsetArray(Acal[1:Ncal,1:Ncal],0:N,0:N)  # coefficient matrix for chi in v Chebyshev
A_chiv = OffsetArray(Acal[1:Ncal,1:Ncal],0:N,0:N)  # coefficient matrix for chi in chi Chebyshev
A_chichi = OffsetArray(Acal[1:Ncal,1:Ncal],0:N,0:N)  # coefficient matrix for chi in chi Chebyshev
B_chichi = OffsetArray(Acal[1:Ncal,1:Ncal],0:N,0:N)  # coefficient matrix for chi in chi Chebyshev
### --- From here, supposing matrix element as col, row as in julia notation (only using these arrays for matrix calc)
b = OffsetArray(zeros(Float64,Ncal),0:N)  # A vector composed of Chebyshev coefficients for the basic state flow
c = OffsetArray(zeros(Float64,Ncal),0:N)  # factorized vector (c_0=2, c_n=1, (n>0))

# setting parameters
α = 1.0
Re = 10000.0

# setting the Chebyshev coefficients for the basic flow
b[0] = 0.5
b[2] = -b[0]

# setting the factorized vector
c[0] = 2.0
c[1:N] .= 1.0

# setting maximum even and odd numbers from N
Nemax = 0
Nomax = 1
for i in 0:2:N
    Nemax = i
end
for i in 1:2:N
    Nomax = i
end
println("Nemax = ", Nemax, ", Nomax = ", Nomax)

# setting matrices
# 支配方程式から構築される部分 (チェビシェフ次数 0<=n<=N-2 の範囲)
# この段階では境界条件で成り立つ係数の関係は考慮しない
for n in 0:N-2
    n2 = n^2
    
    # In v rows, a_{v,n} and a_{chi,n}
    for p in n+2:2:N
        p2 = p^2
        A_vv[n,p] = p * (p2 - n2)
    end
    
    A_vv[n,n] = - c[n] * α^2
    A_vchi[n,n] = - c[n]
    
    # In chi rows, a_{v,n} and a_{chi,n}
    for p in n+2:2:N
        p2 = p^2
        A_chichi[n,p] = p * (p2 - n2)
    end
    
    A_chichi[n,n] = - c[n] * α^2

    for p in -N:N
        abs_n_p = abs(n - p)
        abs_p = abs(p)
        if abs_n_p <= N
            A_chichi[n,abs_p] = A_chichi[n,abs_p] + complex(0.0, - 0.5 * α * Re * c[abs_p] * c[abs_n_p] * b[abs_n_p])
        end

        if abs_n_p+2 <= N
            acoef = 0.0
            for m in abs_n_p+2:2:N
                m2 = m^2
                acoef = acoef + m * (m2 - (n - p)^2) * b[m]
            end
            A_chiv[n,abs_p] = A_chiv[n,abs_p] + complex(0.0, 0.5 * α * Re * c[abs_p] * acoef)
        end
    end
    
    # In chi rows, lambda coefficients
    B_chichi[n,n] = complex(0.0, - α * Re * c[n])

end

# Set boundary conditions
for p in 0:2:N  # even mode
    A_vv[N-1,p] = 1.0
    A_chichi[N-1,p] = 1.0
end
for p in 1:2:N  # even mode
    A_vv[N,p] = 1.0
    A_chichi[N,p] = 1.0
end

# Gather A_v, A_chi, B_chi -> Acal, Bcal
Acal[1:Ncal,1:Ncal] .= A_vv[0:N,0:N]
Acal[1:Ncal,Ncal+1:Ntot] .= A_vchi[0:N,0:N]
Acal[Ncal+1:Ntot,1:Ncal] .= A_chiv[0:N,0:N]
Acal[Ncal+1:Ntot,Ncal+1:Ntot] .= A_chichi[0:N,0:N]
Bcal[Ncal+1:Ntot,Ncal+1:Ntot] .= B_chichi[0:N,0:N]

# Convert Transpose type to Matrix (Array) type (ドット演算して要素だけコピーしている)
A_J .= transpose(Acal)  # A_J = Transpose type
B_J .= transpose(Bcal)  # B_J = Transpose type

F = schur(A_J,B_J)
Dcal = F.α ./ F.β
for i in 1:Ncal
    println("Schur(Eig): ", Dcal[i])
end

#G = eigvecs(Ccal)
#H = inv(G) * Ccal * G
#ncmax = findmax(F)[1]
#Ae = OffsetArray(A[0:div(N,2),0:div(N,2)],0:div(N,2),0:div(N,2))
#Ao = OffsetArray(A[1:div(N,2),1:div(N,2)],1:div(N,2),1:div(N,2))
#Be = OffsetArray(B[0:div(N,2),0:div(N,2)],0:div(N,2),0:div(N,2))
#Bo = OffsetArray(B[1:div(N,2),1:div(N,2)],1:div(N,2),1:div(N,2))
#for n in 0:div(N,2)
#    for p in 0:div(N,2)
#        Ae[n,p] = A[2*n,2*p]
#        Be[n,p] = B[2*n,2*p]
#    end
#end
#for n in 1:div(N,2)
#    for p in 1:div(N,2)
#        Ao[n,p] = A[2*n-1,2*p-1]
#        Bo[n,p] = B[2*n-1,2*p-1]
#    end
#end
#Acal[1:div(N,2)-1,1:div(N,2)-1] .= Ae[0:div(N,2)-2,0:div(N,2)-2]
#Bcal[1:div(N,2)-1,1:div(N,2)-1] .= Be[0:div(N,2)-2,0:div(N,2)-2]
#Ccal = inv(Bcal[1:div(N,2)-1,1:div(N,2)-1]) * Acal[1:div(N,2)-1,1:div(N,2)-1]
##Ccal = inv(Bcal) * Acal
#Dcal = eigvals(Ccal)
#F = imag.(Dcal)
#for i in 1:div(N,2)-1
#    println(F[i])
#end

#Acal[1:div(N,2)-2,1:div(N,2)-2] .= Ao[1:div(N,2)-2,1:div(N,2)-2]
#Bcal[1:div(N,2)-2,1:div(N,2)-2] .= Bo[1:div(N,2)-2,1:div(N,2)-2]
#Ccal = inv(Bcal[1:div(N,2)-2,1:div(N,2)-2]) * Acal[1:div(N,2)-2,1:div(N,2)-2]
##Ccal = inv(Bcal) * Acal
#Dcal = eigvals(Ccal)
#F = imag.(Dcal)
#for i in 1:div(N,2)-2
#    println(F[i])
#end


Nemax = 200, Nomax = 199
Schur(Eig): NaN + NaN*im
Schur(Eig): 0.009959653231081049 - 48468.16070542684im
Schur(Eig): 0.010639510751611096 - 2607.8212490697633im
Schur(Eig): 0.012351539288276479 - 800.590046080902im
Schur(Eig): 0.015015902879698298 - 384.45342326340136im
Schur(Eig): 0.01865670668560613 - 225.024165049837im
Schur(Eig): 0.023254868059602536 - 147.71669819294004im
Schur(Eig): 0.02881645974978412 - 104.4202105849741im
Schur(Eig): 0.035328284168819016 - 77.7782209386429im
Schur(Eig): 0.042790145135120326 - 60.219231986395144im
Schur(Eig): 0.051190030989770624 - 48.043442857050216im
Schur(Eig): 0.06052407372056047 - 39.25389119196648im
Schur(Eig): 0.07078015331861712 - 32.70373530270634im
Schur(Eig): 0.08195179841863179 - 27.69170216772187im
Schur(Eig): 0.09402636094641309 - 23.772156870806974im
Schur(Eig): 0.10699539386551074 - 20.649194542724224im
Schur(Eig): 0.12084564762227566 - 18.121217196096186im
Schur(Eig): 0.13556718775621068 - 16.046282019972388im
Schur(Eig): 0.1511

In [8]:
#-- Drawing
##########
#  Plot  #
##########
using PyPlot

rc("font", family="IPAPGothic")
fig = figure("pyplot_majorminor",figsize=(7,5))

x = real.(Dcal[3:Ncal-4])
y = imag.(Dcal[3:Ncal-4])

ax = gca()

cp = scatter(x,y)

ax = gca()

xlabel("X")
ylabel("Y")
#if i == 1 #div(ne,2)-1
#   plt.colorbar(cp)
#end
grid("on")

fig.canvas.draw()
gcf() # Needed for IJulia to plot inline

#PyPlot.title("Re = Re[i]")

fname = "test.png"
#savefig("B-init_2d.pdf")
savefig(fname)
#psi


Figure(PyObject <Figure size 700x500 with 1 Axes>)

In [6]:
#-- Drawing
##########
#  Plot  #
##########
#using PyPlot

#rc("font", family="IPAPGothic")
#fig = figure("pyplot_majorminor",figsize=(7,5))

nx = Ncal-4
ny = Ncal-4

xax = reshape(zeros(nx,ny),nx,ny)
yax = reshape(zeros(nx,ny),nx,ny)

for i in 1:nx
    xax[i,1:ny] .= i  # xmin + (xmax-xmin) * (i-1) / (nx-1)
end
for i in 1:ny
    yax[1:nx,i] .= i  # ymin + (ymax-ymin) * (i-1) / (ny-1)
#    yax[1:nx,i] .= Re[i]  # ymin + (ymax-ymin) * (i-1) / (ny-1)
end

cont_val = reshape(zeros(nx,ny),nx,ny)
shade_val = reshape(zeros(nx,ny),nx,ny)
#for i in 1:nr#div(nr,2)-1:div(nr,2)+1
    cont_val = imag.(Ccal[1:nx,1:ny])
    #for j in 1:ne
    shade_val = imag.(A_J[1:nx,1:ny])
    #end
#draw_num2 = 2
#if draw_num2 == 1
#    cp = contourf(xax[1:nx,1:nx], xax'[1:nx,1:nx], Pftime[1:nx,1:nx,1], levels=[-15.0, -10.0, -5.0, 0.0, 5.0, 10.0, 15.0, 20.0], origin="image", cmap=ColorMap("viridis"), extend="both")
    #cp = contourf(xax[1:nx,1:nx], xax'[1:nx,1:nx], Pftime[1:nx,1:nx,2], levels=[-2.0, -1.0, -0.5, -0.25, 0.25, 0.5, 1.0, 1.5, 2.0], origin="image", cmap=ColorMap("viridis"), extend="both")
    #cp = contourf(xax[1:nx,1:no], xax'[1:nx,1:no], Pftime[1:nx,1:nx,2]*Hop'*inv(Hop*Pftime[1:nx,1:nx,2]*Hop'+Ro), origin="image", cmap=ColorMap("viridis"), extend="both")
#elseif draw_num2 == 2
    ##cp = contour(xax[1:nx,1:ny], yax[1:nx,1:ny], shade_val[1:nx,1:ny], 8, levels=[-0.01, -0.005, -0.003, -0.002, -0.001, -0.0005, 0.0, 0.0005, 0.001, 0.002, 0.003, 0.005, 0.01])
    cp = contourf(xax[1:nx,1:ny], yax[1:nx,1:ny], cont_val[1:nx,1:ny], cmap=ColorMap("viridis"), extend="both")
    ##cp = contourf(xax[1:nx,1:ny], yax[1:nx,1:ny], cont_val[1:nx,1:ny], 8, levels=[-1.0, -0.5, -0.2, -0.1, 0.0, 0.1, 0.2, 0.5, 1.0], cmap=ColorMap("viridis"), extend="both")
    #cp = contourf(xax[1:nx,1:ny], yax[1:nx,1:ny], shade_val[1:nx,1:ny], 10, cmap=ColorMap("viridis"), extend="both")
#elseif draw_num2 == 3
#    cp = contourf(xoax[1:nx,1:nto-1], toax[1:nx,1:nto-1], evec_max[1:nx,1:nto-1], 10, cmap=ColorMap("viridis"), extend="both")
#elseif draw_num2 == 4
#    cp = contourf(xoax[1:nx,1:40], toax[1:nx,1:40], x_inc[1:nx,1:40], levels=[-2.0, -1.0, -0.5, -0.25, 0.25, 0.5, 1.0, 1.5, 2.0], cmap=ColorMap("viridis"), extend="both")
#elseif draw_num2 == 5
#    cp = contourf(xoax[1:no,1:40], toax[1:no,1:40], y_innov[1:no,1:40], levels=[-2.0, -1.0, -0.5, -0.25, 0.25, 0.5, 1.0, 1.5, 2.0], cmap=ColorMap("viridis"), extend="both")
#end
#ax.label(cp, inline=1, fontsize=10)
#legend()
ax = gca()

xlabel("X")
ylabel("Y")
#if i == 1 #div(ne,2)-1
   plt.colorbar(cp)
#end
grid("on")

PyPlot.title("Re = Re[i]")

#########################
#  Set tick dimensions  #
#########################
#ax.xaxis.set_tick_params(which="major",length=5,width=2,labelsize=10)
#ax.xaxis.set_tick_params(which="minor",length=5,width=2)

fig.canvas.draw() # Update the figure
gcf() # Needed for IJulia to plot inline
#fname = "Re" * string(1) * ".png"
fname = "test.png"
#savefig("B-init_2d.pdf")
#savefig(fname)
#psi
#end
#println(shade_val[1:na,1])

LoadError: UndefVarError: Ccal not defined

In [47]:
a33 = reshape(zeros(Complex{Float64},2,2),2,2)
c33 = reshape(zeros(Complex{Float64},2,2),2,2)
println(a33)
a33[1,1] = 1.0 + 2.0im
a33[2,2] = 2.0 - 1.0im
b33 = transpose(a33)
c33 .= b33
schur(a33,c33)

ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]


GeneralizedSchur{ComplexF64, Matrix{ComplexF64}}
S factor:
2×2 Matrix{ComplexF64}:
 2.23607-0.0im      0.0+0.0im
     0.0+0.0im  2.23607+0.0im
T factor:
2×2 Matrix{ComplexF64}:
 2.23607+0.0im      0.0+0.0im
     0.0+0.0im  2.23607+0.0im
Q factor:
2×2 Matrix{ComplexF64}:
 -0.447214-0.894427im  0.0+0.0im
       0.0+0.0im       1.0+0.0im
Z factor:
2×2 Matrix{ComplexF64}:
 -1.0-0.0im       0.0+0.0im
 -0.0-0.0im  0.894427+0.447214im
α:
2-element Vector{ComplexF64}:
 2.23606797749979 - 0.0im
 2.23606797749979 + 0.0im
β:
2-element Vector{ComplexF64}:
 2.23606797749979 + 0.0im
 2.23606797749979 + 0.0im